## Загрузка данных

### Библиотеки:

In [1]:
import pandas as pd
from openpyxl import load_workbook

from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster.hierarchy import linkage, fcluster

### Файлы

#### ТОП10 выдачи Яндекса по каждой фразе

In [2]:
df = pd.read_excel('sem_yadro_krovelnye_raboty.xlsx', sheet_name = 'serp')

### Проверка на пропуски Nan

#### Подсчет пропусков во всей таблице

In [3]:
df.isna().sum()

Фраза             0
Title [Yandex]    0
URL [Yandex]      0
dtype: int64

#### Удаление строк с пропусками

In [4]:
df.dropna(inplace=True)

In [5]:
df.isna().sum()

Фраза             0
Title [Yandex]    0
URL [Yandex]      0
dtype: int64

### Переименование столбцов

In [6]:
df.rename(columns={'URL [Yandex]':'URL', 'Title [Yandex]':'Title'}, inplace=True)

### Отбрасывание строк, в URL которых есть "yandex"

In [7]:
filt =~(df['URL'].str.contains('yandex', regex=True))

In [8]:
df = df[filt]

## Кластеризация

### Векторизация

#### Создание экземпляра класса векторизатора

In [9]:
vectorizer = CountVectorizer(min_df=2, token_pattern='[^\ ]+')

#### Обучение векторизатора на собранных URL

In [10]:
ml_df = vectorizer.fit_transform(df['URL'])

### Кластеризация

In [11]:
link_ml_df = linkage(ml_df.toarray(), method='ward')

In [12]:
df['Кластер_serp'] = fcluster(link_ml_df, 70, criterion = 'maxclust')

#### Результат

In [13]:
df.head()

,Фраза,Title,URL,Кластер_serp
0,кровельные работы в спб и лен обл,"Кровельные работы, цена за м2 в Ленинградской ...",https://www.remontnik.ru/leningradskaya_oblast...,70
2,кровельные работы в спб и лен обл,"Монтаж кровли в Санкт-Петербурге, кровельные р...",https://len-krov.ru/,36
4,кровельные работы в спб и лен обл,Кровельные работы в Ленинградской области | Кр...,https://krovmontage.spb.ru/krovelnye-raboty-v-...,70
5,кровельные работы в спб и лен обл,ремонт кровли - Предложения услуг в Ленинградс...,https://www.avito.ru/leningradskaya_oblast/pre...,51
6,кровельные работы в спб и лен обл,Кровельные работы Ленинградская / Строительств...,https://uslugio.com/leningradskaya/1/2/kroveln...,70


#### Подсчет количества фраз внутри кластеров

In [14]:
df['Кластер_serp'].value_counts()

70    542
1     339
2     313
3     291
4     252
     ... 
65      8
63      8
68      7
67      7
69      7
Name: Кластер_serp, Length: 70, dtype: int64

#### Удаление определенных кластеров

In [15]:
cluster_filtering = df['Кластер_serp'] != 70
#cluster_filtering = (df['Кластер'] != 70)&(df['Кластер'] != 1)&(df['Кластер'] != 2)

In [16]:
df = df[cluster_filtering]

In [17]:
df['Кластер_serp'].value_counts()

1     339
2     313
3     291
4     252
5     214
     ... 
63      8
65      8
68      7
67      7
69      7
Name: Кластер_serp, Length: 69, dtype: int64

### Экспорт кластеризации

Создание нового листа в исходной таблице:

In [18]:
with pd.ExcelWriter('sem_yadro_krovelnye_raboty.xlsx', engine='openpyxl') as report:
    report.book = load_workbook('sem_yadro_krovelnye_raboty.xlsx')
    df.to_excel(report, 'Кластеризация SERP', index=False)

## Слияние с фразами, кластеризованными по словам

Загрузка таблицы с первого листа с кластеризованными по смыслу словами:

In [19]:
df_clustered_by_words = pd.read_excel('sem_yadro_krovelnye_raboty.xlsx')

Удаление дубликатов фраз в таблице с кластеризованными серпами:

In [21]:
df_clustered_by_serps = df.drop_duplicates(subset='Фраза')

Удаление ненужных столбцов:

In [25]:
df_clustered_by_serps = df_clustered_by_serps[['Фраза','Кластер_serp']]

Слияние двух таблиц:

In [26]:
new_df = df_clustered_by_words.merge(df_clustered_by_serps, on='Фраза')

In [27]:
new_df

,Кластер,Название кластера,Фраза,Ч,"""Ч""","""!Ч""",Кластер_serp
0,КРОВЕЛЬНЫЕ РАБОТЫ,общие запросы,кровельные работы в спб и лен обл,1186,1186,1186,36
1,NaN,общие запросы,кровельные работы в спб,4136,815,815,1
2,NaN,общие запросы,кровельные работы санкт петербург,1742,221,221,2
3,NaN,общие запросы,кровельные работы ленинградская область,162,162,162,7
4,NaN,общие запросы,кровельные работы питер,141,141,141,61
...,...,...,...,...,...,...,...
224,ОНДУЛИН,цена,покрыть крышу ондулином цена работы,170,6,6,57
225,NaN,цена,укладка ондулина цена,61,23,8,56
226,NaN,цена,монтаж ондулина цена,102,34,34,56
227,NaN,цена,монтаж ондулина цена за м2 за работу,13,13,13,67


### Экспорт

Создание нового листа в исходной таблице:

In [29]:
with pd.ExcelWriter('sem_yadro_krovelnye_raboty.xlsx', engine='openpyxl') as report:
    report.book = load_workbook('sem_yadro_krovelnye_raboty.xlsx')
    new_df.to_excel(report, 'ядро+кластер serp', index=False)

## Получение списка доменов конкурентов

### Получение домена из URL и подсчет количества

In [30]:
domain_list = (df['URL']
               .apply(lambda x: x.split('/'))
               .apply(lambda x: x[2])
               .value_counts().reset_index()
               .rename(columns={'index':'Домен','domain':'Видимость'})
              )

In [31]:
domain_list

,Домен,URL
0,spb.krovlyaural.ru,339
1,roof-petersburg.ru,313
2,razvitiee.com,309
3,roof.spb.ru,252
4,mastera-v-spb.ru,214
5,esp-spb.ru,138
6,tosno-zavod.ru,131
7,remont-krovli-spb.ru,111
8,sankt-peterburg.trade-services.ru,110
9,spb.profi.ru,103


### Экспорт списка окнкурентов

In [33]:
with pd.ExcelWriter('sem_yadro_krovelnye_raboty.xlsx', engine='openpyxl') as report:
    report.book = load_workbook('sem_yadro_krovelnye_raboty.xlsx')
    domain_list.to_excel(report, 'Список конкурентов', index=False)